# Artificial Intelligence
## Assignment 3 – Probabilistic Reasoning

### Personal details

* **Name:** Ahmed Jabir Zuhayr

In this assignment, you will learn how to use probabilistic reasoning to predict future events based on given evidence. We will define a Bayesian network and use it to compute the probabilities of queried variables with the help of some initial input.

### 3.0 – Bayesian Networks

Consider an insurance company trying to determine the risk of 1) *an accident* and 2) *theft* of a car. The company has access to information based on input variables provided by the customer, such as whether the driver is young, has been driving for a long time and if they have a garage. All variables have been binarized for simplicity, so there is no scale of membership – the customer is either young (1) or not (0). 

The company has built a **Bayesian network** to model the relationships between these variables and the risk of an accident or theft. Most input variables don't have a *direct* influence on the risk, but affect it *indirectly* through **hidden variables** that must be estimated. The risk is represented by the output variables `Accident` and `Theft`. The Bayesian network is structured as follows:

![bnet](bnet.png)

Here the known input variables (`Young`, `HighIncome`, `ExperiencedDriver`, `HistoryOfAccidents`, `ModernCar`, `Garage`) are represented by green nodes, the hidden variables (`RiskAverse`, `SkilledDriver`, `SafetyFeatures`, `Tempting`, `Antitheft`) by yellow nodes and the output variables (`Accident`, `Theft`) by red nodes. The arrows represent the dependencies between the variables, such that an arrow from node A to node B indicates that A has a direct influence on B.

Since we only have direct access to the input variables, we need to estimate their effect on the unknown variables. In practice, this is usually done empirically by using data from the past, allowing us to make statements like "young drivers tend to be bigger risk-takers" or "modern cars tend to have better safety features but are also more tempting for thieves". These observations are summarized in the following **conditional probability tables** (CPTs):

<br>

<div align="center">

| Young | HighIncome | P(RiskAverse \| Young, HighIncome) |
|:-----:|:----------:|:-------------:|
|   0   |     0      |     0.90      |
|   1   |     0      |     0.40      |
|   1   |     1      |     0.10      |
|   0   |     1      |     0.50      |

<center>Table 1. CPT for risk aversion</center>

<br>

| ExperiencedDriver | HistoryOfAccidents | P(SkilledDriver \| ExperiencedDriver, HistoryOfAccidents) |
|:-----------------:|:------------------:|:----------------:|
|         0         |         0          |      0.40        |
|         1         |         0          |      0.85        |
|         1         |         1          |      0.50        |
|         0         |         1          |      0.10        |

<center>Table 2. CPT for driving skill</center>

<br>

| ModernCar | P(SafetyFeatures \| ModernCar | P(Tempting \| ModernCar) | P(AntiTheft \| ModernCar) |
|:---------:|:-----------------------:|:---------:|:-------------:|
|     0     |       0.20              |    0.40   |     0.30      |
|     1     |       0.75              |    0.80   |     0.90      |

<center>Table 3. CPT for safety features, temptation and anti-theft features</center>

<br>

| RiskAverse | SkilledDriver | SafetyFeatures | P(Accident \| RiskAverse, SkilledDriver, SafetyFeatures) |
|:----------:|:-------------:|:--------------:|:-----------:|
|     0      |      0        |      0         |    0.90     |
|     0      |      0        |      1         |    0.65     |
|     0      |      1        |      0         |    0.35     |
|     0      |      1        |      1         |    0.20     |
|     1      |      0        |      0         |    0.20     |
|     1      |      0        |      1         |    0.10     |
|     1      |      1        |      0         |    0.10     |
|     1      |      1        |      1         |    0.05     |

<center>Table 4. CPT for accident</center>

<br>

| Tempting | AntiTheft | Garage | P(Theft \| Tempting, AntiTheft, Garage) |
|:--------:|:---------:|:------:|:--------:|
|    0     |     0     |   0    |   0.30   |
|    0     |     0     |   1    |   0.15   |
|    0     |     1     |   0    |   0.15   |
|    0     |     1     |   1    |   0.10   |
|    1     |     0     |   0    |   0.80   |
|    1     |     0     |   1    |   0.60   |
|    1     |     1     |   0    |   0.40   |
|    1     |     1     |   1    |   0.30   |

<center>Table 5. CPT for theft</center>

<br>

</div>

We also need the **prior probabilities** for the root nodes of the network, which are given as follows:

<br>

<div align="center">

| Young | HighIncome | ExperiencedDriver | HistoryOfAccidents | ModernCar | Garage |
|:-----:|:----------:|:----------------:|:------------------:|:---------:|:------:|
| 0.20  |   0.30     |      0.60        |       0.15         |   0.40    |  0.25  |

<center>Table 6. Prior probabilities for root nodes</center>

</div>


**Task 1: Setting up the Bayesian network (0.2 pt)**

Now let's begin our implementation of the Bayesian network. The necessary classes are defined in `ex3_utils.py`. 

Most of this has already been implemented for you, but you will need to fill in the missing parts.

In [1]:
from ex3_utils import Variable, BayesNet

Let's start by creating some variables. We simply copy the CPTs from above.

In [4]:
# Input variables
# Format: Variable(name, probability)
Young = Variable("Young", 0.20)
HighIncome = Variable("HighIncome", 0.30)
ExperiencedDriver = Variable("ExperiencedDriver", 0.60)
HistoryOfAccidents = Variable("HistoryOfAccidents", 0.15)
ModernCar = Variable("ModernCar", 0.40)
Garage = Variable("Garage", 0.25)

# Hidden and output variables
# Format: Variable(name, {cpt}, [parents])
RiskAverse = Variable("RiskAverse", {(0, 0): 0.90, (1, 0): 0.40, (1, 1): 0.10, (0, 1): 0.50}, [Young, HighIncome])
# ---------- YOUR CODE HERE ----------- #
SkilledDriver = Variable("SkilledDriver", {(0, 0): 0.40, (1, 0): 0.85, (1, 1): 0.50, (0, 1): 0.10}, [ExperiencedDriver, HistoryOfAccidents])
# ---------- YOUR CODE HERE ----------- #
SafetyFeatures = Variable("SafetyFeatures", {0: 0.20, 1: 0.75}, [ModernCar])
Tempting = Variable("Tempting", {0: 0.40, 1: 0.80}, [ModernCar])
AntiTheft = Variable("AntiTheft", {0: 0.30, 1: 0.90}, [ModernCar])
Accident = Variable("Accident", {
    (0, 0, 0): 0.90, (0, 0, 1): 0.65,
    (0, 1, 0): 0.35, (0, 1, 1): 0.20,
    (1, 0, 0): 0.20, (1, 0, 1): 0.10,
    (1, 1, 0): 0.10, (1, 1, 1): 0.05
}, [RiskAverse, SkilledDriver, SafetyFeatures])
# ---------- YOUR CODE HERE ----------- #
Theft = Variable("Theft", {
    (0, 0, 0): 0.30, (0, 0, 1): 0.15,
    (0, 1, 0): 0.15, (0, 1, 1): 0.10,
    (1, 0, 0): 0.80, (1, 0, 1): 0.60,
    (1, 1, 0): 0.40, (1, 1, 1): 0.30
}, [Tempting, AntiTheft, Garage])
# ---------- YOUR CODE HERE ----------- #

We also need a function for calculating the probability of a variable taking a specific value given some relevant evidence.

In [5]:
def P(variable, query, evidence=None):
    """
    Computes the probability of a variable taking a specific value given some evidence.
    If no evidence is provided, returns the prior probability.
    Considers only direct parents of the variable for conditional probabilities.
    """
    if evidence is None or variable.parents is None:
        return variable.cpt[query] # P(variable=query)
    if len(variable.parents) > 1:
        evidence_query = tuple(evidence[parent] for parent in variable.parents)
    else:
        evidence_query = evidence[variable.parents[0]]
    return variable.cpt[evidence_query][query] # P(variable=query | evidence)

In [6]:
result1 = P(ModernCar, 0)
print(f"P(-ModernCar): {result1:.2f}")
result2 = P(RiskAverse, 1, {Young: 1, HighIncome: 1})
print(f"P(+RiskAverse | +Young, +HighIncome): {result2:.2f}")

P(-ModernCar): 0.60
P(+RiskAverse | +Young, +HighIncome): 0.10


In [14]:
print("The probability of someone being a non-skilled driver despite being experienced and having no history of accidents is: ", end="")
# ---------- YOUR CODE HERE ----------- #
result3 = P(SkilledDriver, 0, {ExperiencedDriver: 1, HistoryOfAccidents: 0})
# ---------- YOUR CODE HERE ----------- #
print(f"{result3:.2f}")

The probability of someone being a non-skilled driver despite being experienced and having no history of accidents is: 0.15


Now we're ready to define our Bayesian network.

In [15]:
bnet = BayesNet()
bnet.add_node(Young)
bnet.add_node(HighIncome)
bnet.add_node(ExperiencedDriver)
bnet.add_node(HistoryOfAccidents)
bnet.add_node(ModernCar)
bnet.add_node(Garage)
bnet.add_node(RiskAverse)
bnet.add_node(SkilledDriver)
bnet.add_node(SafetyFeatures)
bnet.add_node(Tempting)
bnet.add_node(AntiTheft)
bnet.add_node(Accident)
bnet.add_node(Theft)

The `BayesNet` class includes two methods that you might need in the next task:

In [16]:
print("BayesNet created with nodes:")
print(bnet.nodes)
print("BayesNet is empty:", bnet.is_empty())
print("Removing node 'Garage' from the net")
bnet_rest = bnet.get_rest(remove=Garage)
print("BayesNet after removal:")
print(bnet_rest.nodes)

BayesNet created with nodes:
[<ex3_utils.Variable object at 0x78abe8ac5bd0>, <ex3_utils.Variable object at 0x78ac0020d910>, <ex3_utils.Variable object at 0x78abe8d8e950>, <ex3_utils.Variable object at 0x78abe8d8ed50>, <ex3_utils.Variable object at 0x78abe8e066d0>, <ex3_utils.Variable object at 0x78abe8e06900>, <ex3_utils.Variable object at 0x78abe8e03f50>, <ex3_utils.Variable object at 0x78abe8ace390>, <ex3_utils.Variable object at 0x78abe8aceff0>, <ex3_utils.Variable object at 0x78abe8ace690>, <ex3_utils.Variable object at 0x78abe8acecf0>, <ex3_utils.Variable object at 0x78abe8acddf0>, <ex3_utils.Variable object at 0x78abe8ace210>]
BayesNet is empty: False
Removing node 'Garage' from the net
BayesNet after removal:
[<ex3_utils.Variable object at 0x78abe8ac5bd0>, <ex3_utils.Variable object at 0x78ac0020d910>, <ex3_utils.Variable object at 0x78abe8d8e950>, <ex3_utils.Variable object at 0x78abe8d8ed50>, <ex3_utils.Variable object at 0x78abe8e066d0>, <ex3_utils.Variable object at 0x78abe8

The class isn't much more than a list of `Variable` objects. Most functionality is implemented with the help of the `parents` attributes of the `Variable` class.

**Task 2: Inference by enumeration (0.8 pt)**

The problem is as follows: given some evidence, what is the probability of an accident or theft?

With the variables defined as they are, we can query the probability of an event by using the `P` function as before. For example, to calculate the probability of a theft if the car is tempting, doesn't have anti-theft features and isn't housed in a garage, we could in theory write:

In [17]:
result4 = P(Theft, 1, {Tempting: 1, AntiTheft: 0, Garage: 0})
print(f"P(+Theft | +Tempting, -AntiTheft, -Garage): {result4:.2f}")

P(+Theft | +Tempting, -AntiTheft, -Garage): 0.80


However, this is not the sort of information we generally have access to. We don't know if the car is tempting or not, or if it has anti-theft features. That is why we need to *enumerate* over all the possible values of the variables that are not known, i.e., the variables that are not included in the evidence set.

Your next and final task is to implement the `enumeration_ask` and `enumerate_all` functions. You will need to make use of *dictionaries*, which you can brush up on e.g. in the [Python documentation](https://docs.python.org/3/tutorial/datastructures.html#dictionaries).

(Hint: pseudocode for the algorithms can be found in the lecture slides or the textbook in chapter 13.3: *Exact Inference in Bayesian Networks*, though the implementation details may differ slightly.)

In [25]:
def normalize(Q_x):
    """
    Normalize the distribution Q_x so that the sum of its values equals 1.
    """
    total = sum(Q_x.values())
    return {k: v / total for k, v in Q_x.items()}

def enumerate_all(bnet, e):
    # ---------- YOUR CODE HERE ----------- #
    # 1. Base case: if the Bayesian network is empty, return 1.0
    if bnet.is_empty():
        return 1.0
    
    # 2. Get the first variable V in the Bayesian network and the rest of the variables
    V = bnet.nodes[0]
    rest = bnet.get_rest(remove=V)

    # 3. If V is in the evidence, return its probability multiplied with a recursive call for the rest of the variables
    if V in e:
        return P(V, e[V], e) * enumerate_all(rest, e)
    
    # 4. If V is not in the evidence...
    total = 0.0
    for v in [0, 1]:
        # 5. Create two copies of the evidence, each one extended with a different value for V (0 and 1)
        e_extended = e.copy()
        e_extended[V] = v
        # 6. Return the sum of the probabilities of V for each value multiplied by recursive calls for the rest of the variables
        total += P(V, v, e_extended) * enumerate_all(rest, e_extended)
    return total

    # ---------- YOUR CODE HERE ----------- #

def enumeration_ask(X, e, bnet):
    # ---------- YOUR CODE HERE ----------- #
    Q_x = {}
    # 1. Create two copies of the evidence, each one extended with a different value for X (0 and 1)
    for x in [0, 1]:
        e_extended = e.copy()
    # 2. Initialize an empty dictionary to hold the probabilities for each value of X
        e_extended[X] = x

    # 3. Compute the probabilities for each value of X using the enumerate_all function twice
        Q_x[x] = enumerate_all(bnet, e_extended)

    # 4. Return the normalized probabilities
    return normalize(Q_x)

    # ---------- YOUR CODE HERE ----------- #

Note that following the convention of the textbook, `enumeration_ask` does not support querying for a specific value of a variable like our `P` function, but rather always returns the probability distribution over both possible values in our binary case.

All of the following code should execute without errors.

In [26]:
result5 = enumeration_ask(Accident, {Young: 1, HighIncome: 1, ExperiencedDriver: 0, HistoryOfAccidents: 1, ModernCar: 0, Garage: 0}, bnet)
result6 = enumeration_ask(Accident, {Young: 0, HighIncome: 0, ExperiencedDriver: 1, HistoryOfAccidents: 0, ModernCar: 1, Garage: 0}, bnet)
result7 = enumeration_ask(Accident, {Young: 0, HighIncome: 0, ExperiencedDriver: 1, HistoryOfAccidents: 0, ModernCar: 1, Garage: 1}, bnet)
result8 = enumeration_ask(Accident, {Young: 1, HighIncome: 1}, bnet)
result9 = enumeration_ask(Accident, {Young: 0, ExperiencedDriver: 1, HighIncome: 0, HistoryOfAccidents: 1, ModernCar: 0, Garage: 0}, bnet)
result10 = enumeration_ask(ModernCar, {Accident: 1}, bnet)
result11 = enumeration_ask(Theft, {ModernCar: 1, Garage: 0}, bnet)
result12 = enumeration_ask(Theft, {ModernCar: 0, Garage: 0, Young: 1}, bnet)
result12 = enumeration_ask(Theft, {ModernCar: 0, Garage: 0, Young: 0}, bnet)

def format_result(result):
    return {k: round(v, 3) for k, v in result.items()}

print(f"Accident risk for worst possible combination of input variables: {format_result(result5)}")
print(f"Accident risk for best possible combination of input variables: {format_result(result6)}")
print(f"Whether you have a garage or not is irrelevant for accident risk: {format_result(result7)}")
print(f"Missing input – we only know that the driver is young and has high income: {format_result(result8)}")
print(f"Inverse query – if an accident happens, how likely is it that a modern car was involved? {format_result(result10)}")
print(f"Theft risk for a modern car that is not garaged: {format_result(result11)}")
print(f"Theft risk for an old car that is not garaged: {format_result(result12)}")
print(f"Driver age is irrelevant for theft risk: {format_result(result12)}")

Accident risk for worst possible combination of input variables: {0: 0.266, 1: 0.734}
Accident risk for best possible combination of input variables: {0: 0.904, 1: 0.096}
Whether you have a garage or not is irrelevant for accident risk: {0: 0.904, 1: 0.096}
Missing input – we only know that the driver is young and has high income: {0: 0.557, 1: 0.443}
Inverse query – if an accident happens, how likely is it that a modern car was involved? {0: 0.662, 1: 0.338}
Theft risk for a modern car that is not garaged: {0: 0.615, 1: 0.385}
Theft risk for an old car that is not garaged: {0: 0.575, 1: 0.425}
Driver age is irrelevant for theft risk: {0: 0.575, 1: 0.425}


In [33]:
# ---------- YOUR CODE HERE (OPTIONAL) ----------- #
# You can add more queries here if you're interested
result13 = enumeration_ask(Accident, {Garage: 0}, bnet)
print(f"How does not having a garage affect accident risk? {format_result(result13)}")
result14 = enumeration_ask(Accident, {SkilledDriver: 0, SafetyFeatures: 1}, bnet)
print(f"If safety features are good, does driver skill even matter for accidents? {format_result(result14)}")
result15 = enumeration_ask(SafetyFeatures, {Accident: 1}, bnet)
print(f"If an accident happens, how likely were the safety features poor? {format_result(result15)}")
# ---------- YOUR CODE HERE (OPTIONAL) ----------- #

How does not having a garage affect accident risk? {0: 0.775, 1: 0.225}
If safety features are good, does driver skill even matter for accidents? {0: 0.727, 1: 0.273}
If an accident happens, how likely were the safety features poor? {0: 0.695, 1: 0.305}


### EXTRA: Discussion

**Comment on the quality of the predictions of our Bayesian network. Can you think of possible improvements to the model?**

The Bayesian Network successfully captures several important causal relationships, which seems to match with intuitions (and of course the given real data). However, Many important factors are reduced to yes/no: “Skilled driver” vs levels of skill, “Modern car” vs model year or safety rating, “Accident” without severity etc. This limits expressiveness and can hide important nonlinear effects.

We can also improve the model by adding more attributes such as:
- Weather and road conditions
- Driving frequency and exposure
- Traffic density
- Driver fatigue, distraction, or intoxication
- Neighborhood crime rate (for theft)

## Aftermath

Please provide short answers to the following questions:

**1. Did you experience any issues or find anything particularly confusing?**

YOUR ANSWER HERE

**2. Is there anything you would like to see improved in the assignment?**

YOUR ANSWER HERE

### Submission

1. Make sure you have completed all tasks and filled in your personal details at the top of this notebook.
2. Ensure all the code runs without errors: restart the kernel and run all cells in order.
3. Submit *only* this notebook (`ex3.ipynb`) on Moodle.
